In [1]:
cd '/home/psadmin/work/gradle/gradle-server-acceptance/cleansing'

/home/psadmin/work/gradle/gradle-server-acceptance/cleansing


In [7]:
import re
import os
import math
import logging
from enum import Enum
from datetime import datetime
from abc import ABCMeta, abstractmethod
import json
import numpy as np
import pandas as pd
import dataset as ds
from dataset.types import Types
from getconfig_cleansing.util import Util
from getconfig_cleansing.inventory.info import InventoryInfo
from getconfig_cleansing.merge_master import MergeMaster


In [8]:
hosts = pd.read_csv('tests/resources/transfer/host_list.csv')
ports = pd.read_csv('tests/resources/transfer/port_list.csv')
arp_tables = pd.read_csv('tests/resources/transfer/arp_list.csv')
hosts
arp_tables

,Unnamed: 0,スイッチ名,ポート名,IP,MACアドレス,ベンダー
0,0,router1,LAN3,192.168.10.100,00:1b:21:12:34:56,Intel Corporate
1,1,router1,LAN3,192.168.10.1,00:0c:29:12:34:56,"VMware, Inc."
2,2,router1,LAN1,192.168.0.17,10:02:b5:12:34:56,Intel Corporate
3,3,router1,LAN3,192.168.10.4,00:0c:29:12:34:56,"VMware, Inc."
4,4,router1,LAN1,192.168.0.20,00:0c:29:12:34:56,"VMware, Inc."
5,5,router1,LAN1,192.168.0.27,64:b5:c6:12:34:56,"Nintendo Co.,Ltd"
6,6,router1,LAN1,192.168.0.18,2c:fd:a1:12:34:56,ASUSTek COMPUTER INC.
7,7,router1,LAN1,192.168.0.19,bc:5f:f4:12:34:56,ASRock Incorporation
8,8,router1,LAN1,192.168.0.16,00:0c:29:12:34:56,"VMware, Inc."
9,9,router1,LAN1,192.168.0.11,00:0c:29:12:34:56,"VMware, Inc."


In [10]:
df = MergeMaster().join_by_host(hosts, arp_tables, 'IP')

Index(['Unnamed: 0', 'No', 'ホスト名', 'ドメイン', '接続IP', 'システム名', 'モデル名', 'ディスク構成',
       'ネットワーク構成', '検査成績', '検査結果', '備考', 'getconfig_name', 'getconfig_project',
       'ホストグループ', 'テンプレート', '接続アドレス', 'ステータス', '利用可能', 'Syslog監視', 'トリガー',
       'OS名', 'アーキテクチャ', 'CPU数', 'MEM容量', '管理LAN', 'index', 'Test', 'test_id',
       'test_name', 'device', 'value', 'OSリリース'],
      dtype='object')

In [11]:
df = MergeMaster().join_by_host(hosts, ports, 'ホスト名')
df

,Unnamed: 0,No,ホスト名,ドメイン,接続IP,システム名,モデル名,ディスク構成,ネットワーク構成,検査成績,...,管理LAN,index,Test,test_id,test_name,device,value,OSリリース,IP,AdminIP
0,0,1,eternus1,Eternus,10.20.2.52,DX60S3#2,ET063A,"[RAID5 2236416MB:4, RAID5 1118208MB:1]","[MNT:10.20.2.52, RMT:192.168.1.52]",100.0 %,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.20.2.52,False
1,0,1,eternus1,Eternus,10.20.2.52,DX60S3#2,ET063A,"[RAID5 2236416MB:4, RAID5 1118208MB:1]","[MNT:10.20.2.52, RMT:192.168.1.52]",100.0 %,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.168.1.52,False
2,1,1,ostrich,Zabbix,192.168.10.1,NaN,NaN,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",75.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,192.168.10.1,False
3,1,1,ostrich,Zabbix,192.168.10.1,NaN,NaN,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",75.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,192.168.10.4,False
4,1,1,ostrich,Zabbix,192.168.10.1,NaN,NaN,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",75.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,192.168.1.148,True
5,1,1,ostrich,Zabbix,192.168.10.1,NaN,NaN,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",75.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10.154.20.54,True
6,2,2,cent7,Zabbix,192.168.0.20,NaN,NaN,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",75.0 %,...,No data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.168.0.20,False
7,2,2,cent7,Zabbix,192.168.0.20,NaN,NaN,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",75.0 %,...,No data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.17.0.1,False
8,3,3,win2012,Zabbix,192.168.0.24,NaN,NaN,[C:40],[192.168.0.24],50.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,192.168.0.24,False
9,3,3,win2012,Zabbix,192.168.0.24,NaN,NaN,[C:40],[192.168.0.24],50.0 %,...,No data,0.0,NaN,NaN,NaN,NaN,NaN,NaN,192.168.0.14,False


In [13]:
df2 = MergeMaster().join_by_host(df, arp_tables, 'IP')
df2[df2['ホスト名']=='ostrich'].T

,2,3,4,5
Unnamed: 0,1,1,1,1
No,1,1,1,1
ホスト名,ostrich,ostrich,ostrich,ostrich
ドメイン,Zabbix,Zabbix,Zabbix,Zabbix
接続IP,192.168.10.1,192.168.10.1,192.168.10.1,192.168.10.1
システム名,NaN,NaN,NaN,NaN
モデル名,NaN,NaN,NaN,NaN
ディスク構成,"[/boot:500M, /:46.5G, [SWAP]:3G]","[/boot:500M, /:46.5G, [SWAP]:3G]","[/boot:500M, /:46.5G, [SWAP]:3G]","[/boot:500M, /:46.5G, [SWAP]:3G]"
ネットワーク構成,"[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:..."
検査成績,75.0 %,75.0 %,75.0 %,75.0 %
